# 3D Reconstruction

---

## Pseudocode

#### input:
* z_comps: a collection of components for all the z slice

#### pseudocode:

for each set of components for each z slice:
    for each component in this set:
    1. If nearest centroids in z planes above and below the current plane are within a specified x–y radius, 
        then current centroid is a part of that blob, so put it in that blob's collection
    
Compute Centroid of all centroids associated with a blob

return list of blob centroids

In [26]:
# ```
# for comps in z_comps:
#     # comps is a list of components for a single z slice
#     for component in comps:
#         if (nearest centroids in z planes above and below the current plane are within a specified x–y radius):
#             current centroid is a part of that blob, so put it in that blob's collection
# return unique_blobs
        
# ```

In [1]:
%matplotlib inline
# from __future__ import division

import matplotlib.pyplot as plt
# from skimage import data
# from skimage.filters import threshold_otsu, threshold_local
# from skimage.morphology import disk, binary_erosion, binary_opening, label
from skimage.measure import regionprops

import numpy as np

from tifffile import imsave, imread

import pickle

In [2]:
z_comps = pickle.load(open("z_comps.pkl", "rb"))
print(len(z_comps))

100


In [27]:
# Helper functions

def comp_hash(z, comp):
#     hash_key = 17;
#     hash_key = hash_key * 31 + int(round(comp.centroid[0]));
#     hash_key = hash_key * 31 + int(round(comp.centroid[1]));
#     hash_key = hash_key * 31 + z;
    hash_key = (z, int(round(comp.centroid[0])), int(round(comp.centroid[1])))
    return hash_key

def get_min_dist(cc_comp, comps):
    min_dist = float("inf")
    min_comp = None
    for comp in comps:
        euc_dist = np.linalg.norm(np.asarray(cc_comp.centroid) - np.asarray(comp.centroid))
        if (euc_dist < min_dist):
            min_dist = euc_dist
            min_comp = comp   
            
    return min_dist, min_comp

def get_3d_centroid(points):
    zs = [p[0] for p in points]
    ys = [p[1] for p in points]
    xs = [p[2] for p in points]
    z = int(round(float(sum(zs)) / len(points)))
    y = int(round(float(sum(ys)) / len(points)))
    x = int(round(float(sum(xs)) / len(points)))
    centroid = [z, y, x]
    return centroid

In [4]:
# {comp_hash_key : blob_number}
blobs = {}

# {blob_number : blob_num_z_slices}
blob_z_size = {}

blob_num = 0

for k in range(2, len(z_comps) - 2):
    print('z: %d' % k)
    below_comps = z_comps[k - 1]
    above_comps = z_comps[k + 1]
    for curr_comp in z_comps[k]:
        min_dist_below, min_comp_below = get_min_dist(curr_comp, below_comps)
        min_dist_above, min_comp_above = get_min_dist(curr_comp, above_comps)
        if (min_dist_below < 5 and min_dist_above < 5):
            # if the the blob is in the z slice below and the blob is in the z slice above
            hash_key = comp_hash(k, curr_comp)
            hash_below = comp_hash(k - 1, min_comp_below)
            if hash_below in blobs:
                # Blob below is already in the blobs dict.
                blob_num_below = blobs[hash_below]
                blobs[hash_key] = blob_num_below
                blob_z_size[blob_num_below] += 1
            else:
                # New blob detected
                blobs[hash_key] = blob_num
                blob_num += 1
                blob_z_size[blobs[hash_key]] = 1

z: 2
z: 3
z: 4
z: 5
z: 6
z: 7
z: 8
z: 9
z: 10
z: 11
z: 12
z: 13
z: 14
z: 15
z: 16
z: 17
z: 18
z: 19
z: 20
z: 21
z: 22
z: 23
z: 24
z: 25
z: 26
z: 27
z: 28
z: 29
z: 30
z: 31
z: 32
z: 33
z: 34
z: 35
z: 36
z: 37
z: 38
z: 39
z: 40
z: 41
z: 42
z: 43
z: 44
z: 45
z: 46
z: 47
z: 48
z: 49
z: 50
z: 51
z: 52
z: 53
z: 54
z: 55
z: 56
z: 57
z: 58
z: 59
z: 60
z: 61
z: 62
z: 63
z: 64
z: 65
z: 66
z: 67
z: 68
z: 69
z: 70
z: 71
z: 72
z: 73
z: 74
z: 75
z: 76
z: 77
z: 78
z: 79
z: 80
z: 81
z: 82
z: 83
z: 84
z: 85
z: 86
z: 87
z: 88
z: 89
z: 90
z: 91
z: 92
z: 93
z: 94
z: 95
z: 96
z: 97


In [5]:
print('num blobs detected: %d' % blob_num)

num blobs detected: 85


In [6]:
print(blobs)

{(6, 588, 103): 4, (92, 232, 401): 74, (31, 688, 74): 35, (16, 185, 170): 24, (97, 746, 705): 83, (81, 390, 540): 69, (7, 906, 113): 19, (92, 808, 585): 72, (6, 455, 935): 16, (69, 551, 815): 66, (48, 329, 367): 48, (47, 546, 387): 51, (75, 984, 92): 67, (10, 338, 624): 21, (59, 531, 184): 61, (12, 332, 919): 22, (77, 985, 95): 67, (90, 810, 589): 72, (26, 234, 594): 28, (53, 658, 147): 52, (5, 751, 351): 14, (33, 394, 671): 40, (33, 407, 374): 41, (2, 858, 727): 6, (37, 618, 619): 45, (52, 323, 360): 54, (4, 588, 100): 4, (29, 407, 399): 30, (5, 766, 78): 8, (2, 444, 140): 2, (33, 765, 495): 36, (52, 659, 147): 52, (34, 688, 76): 35, (31, 765, 495): 36, (42, 839, 15): 47, (85, 403, 23): 71, (4, 807, 576): 9, (58, 803, 484): 58, (33, 688, 77): 35, (32, 768, 493): 36, (5, 387, 235): 13, (68, 552, 814): 66, (28, 552, 218): 32, (54, 861, 570): 55, (11, 332, 919): 22, (94, 505, 561): 76, (41, 424, 140): 46, (60, 655, 168): 57, (47, 328, 367): 48, (49, 660, 148): 52, (2, 77, 818): 0, (2, 45

In [28]:
blob_centroids = []
for blob_id in range(0, blob_num):
    curr_blob_centroids = []
    for key, val in blobs.iteritems():
        # get all centroids associated with blob_id
        if (blob_id == val):
            curr_blob_centroids.append([key[0], key[1], key[2]])
    blob_centroids.append(get_3d_centroid(curr_blob_centroids))

In [29]:
print(len(blob_centroids))

85


In [30]:
print(blob_centroids)

[[2, 77, 818], [2, 240, 62], [2, 444, 140], [3, 458, 49], [4, 588, 99], [2, 839, 320], [4, 858, 725], [3, 929, 322], [5, 768, 78], [4, 807, 576], [5, 836, 961], [5, 884, 199], [6, 334, 915], [6, 388, 236], [6, 751, 349], [6, 73, 816], [6, 455, 935], [6, 615, 435], [6, 813, 860], [7, 906, 113], [10, 156, 664], [10, 338, 624], [12, 332, 919], [16, 63, 188], [17, 187, 172], [16, 255, 226], [17, 264, 14], [16, 612, 970], [27, 234, 595], [27, 337, 474], [28, 406, 402], [26, 414, 647], [27, 553, 217], [29, 602, 777], [32, 423, 134], [33, 688, 76], [32, 766, 494], [33, 502, 356], [33, 311, 616], [34, 335, 478], [34, 394, 671], [34, 408, 375], [33, 500, 468], [34, 699, 495], [37, 386, 680], [37, 618, 619], [41, 424, 140], [42, 839, 15], [48, 329, 367], [48, 493, 729], [48, 528, 876], [48, 547, 388], [50, 659, 148], [47, 669, 675], [52, 323, 361], [53, 861, 570], [53, 598, 833], [61, 654, 166], [58, 803, 484], [59, 319, 368], [59, 402, 601], [61, 531, 182], [60, 554, 330], [62, 512, 691], [61, 

In [31]:
np.savetxt("blob_centroids.csv", blob_centroids, fmt='%d', delimiter=",")

# Testing function

In [19]:
from detector import reconstruction_3d

In [20]:
b_c = reconstruction_3d(z_comps)

z: 2
z: 3
z: 4
z: 5
z: 6
z: 7
z: 8
z: 9
z: 10
z: 11
z: 12
z: 13
z: 14
z: 15
z: 16
z: 17
z: 18
z: 19
z: 20
z: 21
z: 22
z: 23
z: 24
z: 25
z: 26
z: 27
z: 28
z: 29
z: 30
z: 31
z: 32
z: 33
z: 34
z: 35
z: 36
z: 37
z: 38
z: 39
z: 40
z: 41
z: 42
z: 43
z: 44
z: 45
z: 46
z: 47
z: 48
z: 49
z: 50
z: 51
z: 52
z: 53
z: 54
z: 55
z: 56
z: 57
z: 58
z: 59
z: 60
z: 61
z: 62
z: 63
z: 64
z: 65
z: 66
z: 67
z: 68
z: 69
z: 70
z: 71
z: 72
z: 73
z: 74
z: 75
z: 76
z: 77
z: 78
z: 79
z: 80
z: 81
z: 82
z: 83
z: 84
z: 85
z: 86
z: 87
z: 88
z: 89
z: 90
z: 91
z: 92
z: 93
z: 94
z: 95
z: 96
z: 97


In [21]:
print(b_c == blob_centroids)

True
